In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#import seaborn as sns
import numpy as np

from fiber.cohort import Cohort
from fiber.condition import Procedure, Diagnosis, Drug, VitalSign
from fiber.database.hana import engine, Session, print_sqla

from fiber.database.table import fact

In [ ]:
min_age = fact.AGE_IN_DAYS > 365 * 18

hs = Procedure(code='35.%').with_(min_age) | Procedure(code='36.1%').with_(min_age)
aki = Diagnosis(code='584.9', context='ICD-9')


In [ ]:
heart_surgery_cohort = Cohort(hs)
# vitals, lab, meds = heart_surgery_cohort.get(VitalSign(), LabValues(), Drugs())

In [ ]:
len(heart_surgery_cohort)

In [ ]:
fent, morph = Drug('%Fentanyl%'), Drug('%Morphine%')
hs_fent, hs_morph = (
    Cohort(hs & fent),
    Cohort(hs & morph),
)
len(hs_fent.mrns() - hs_morph.mrns())

In [ ]:
heart_surgery_cohort.exclude(['2241492414','2117141414']).mrns()

In [ ]:
bp = heart_surgery_cohort.get(VitalSign('Systolic Blood Pressure'))

In [ ]:
bp

In [ ]:
co_occurrence = heart_surgery_cohort.occurs((
    aki,
    Diagnosis(code='433.%'),
    Diagnosis(code='434.%'),
    Diagnosis(code='436.%'),
), relative_to=hs)

In [ ]:
co_occurrence

In [ ]:
# heart_surgery_cohort.get(VitalSign('%Blood Pressure'))

In [ ]:
heart_surgery_cohort.lab_results.TEST_NAME.value_counts()

In [ ]:
crea = heart_surgery_cohort.lab_results_for('CREATININE-SERUM')

In [ ]:
develops_aki = co_occurrence[
    (co_occurrence.target_context.str.contains('584')) & (co_occurrence.occurs_after_days < 0)
].medical_record_number.unique()

develops_stroke = co_occurrence[
    (co_occurrence.target_context.str.contains('43')) & (co_occurrence.occurs_after_days < 0)
].medical_record_number.unique()

In [ ]:
crea['develops_aki'] = crea.MEDICAL_RECORD_NUMBER.isin(develops_aki)
crea['develops_stroke'] = crea.MEDICAL_RECORD_NUMBER.isin(develops_stroke)

In [ ]:
crea = crea[crea.VALUE < 10]

_, bins = np.histogram(crea['VALUE'])
g = sns.FacetGrid(crea, hue="develops_aki", height=4, aspect=1)
g = g.map(sns.distplot, "VALUE", bins=bins, hist=False)
